In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import models
import emergent_constraint as ec

In [3]:
model_input_lists = []
for c in ['ocean_constraint', 'total_constraint']:
    for key in ['model_tracer_list',]: 
        model_input_lists.extend([t[0] for t in ec.get_model_tracer_lists(c)[key]])

model_list = sorted(list(set(model_input_lists)))
model_list

['CAMSv20r1',
 'CT2017',
 'CT2019B',
 'CTE2018',
 'CTE2020',
 'MIROC',
 'TM5-Flux-m0f',
 'TM5-Flux-mmf',
 'TM5-Flux-mrf',
 'TM5-Flux-mwf',
 's99oc_ADJocI40S_v2020',
 's99oc_SOCCOM_v2020',
 's99oc_v2020']

In [4]:
from ncar_jobqueue import NCARCluster
try:
    cluster
except:
    cluster = NCARCluster(memory='100GB')
    cluster.scale(32)
cluster

In [5]:
from dask.distributed import Client
try: 
    client
except: 
    client = Client(cluster) # Connect this local process to remote workers
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.42:46460,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [6]:
model_objs = {}
for name in model_list:
    model_objs[name] = models.Model(name)
model_objs    

{'CAMSv20r1': <models.core.Model at 0x2b6cdd029c90>,
 'CT2017': <models.core.Model at 0x2b6cd7a4acd0>,
 'CT2019B': <models.core.Model at 0x2b6cd7a4af90>,
 'CTE2018': <models.core.Model at 0x2b6cd7a4ad50>,
 'CTE2020': <models.core.Model at 0x2b6cd7a4ae50>,
 'MIROC': <models.core.Model at 0x2b6cd7a4a5d0>,
 'TM5-Flux-m0f': <models.core.Model at 0x2b6cd7a4a9d0>,
 'TM5-Flux-mmf': <models.core.Model at 0x2b6cdcc9df50>,
 'TM5-Flux-mrf': <models.core.Model at 0x2b6cdcc9dc50>,
 'TM5-Flux-mwf': <models.core.Model at 0x2b6cdcc9dcd0>,
 's99oc_ADJocI40S_v2020': <models.core.Model at 0x2b6cd4565650>,
 's99oc_SOCCOM_v2020': <models.core.Model at 0x2b6cd45654d0>,
 's99oc_v2020': <models.core.Model at 0x2b6cdc4e9250>}

In [7]:
lat_ranges = [
    (-90., -30.), (-90., -35.), (-90., -40), (-90., -45.), (-90., -50.), (-90., -55.), 

]
lat_ranges = [(-90., x) for x in np.arange(-75., -14., 1)] + [(-45., -15.), (-45., -30.)]

In [8]:
%%time
dsets = {}
for name in model_list:
    print(name)    
    for lat_range in lat_ranges:
        dsets[name] = model_objs[name].open_derived_dataset(
                    'flux_ts_monthly', 
                    lat_range=lat_range,
                    persist=True,
                    clobber=False,
                    return_none=True,
                )

CAMSv20r1
CT2017
CT2019B
CTE2018
CTE2020
MIROC
TM5-Flux-m0f
TM5-Flux-mmf
TM5-Flux-mrf
TM5-Flux-mwf
s99oc_ADJocI40S_v2020
s99oc_SOCCOM_v2020
s99oc_v2020
CPU times: user 26.9 ms, sys: 12 ms, total: 38.9 ms
Wall time: 131 ms


In [9]:
%%time
dsets = {}
for name in model_list:
    print(name)    
    dsets[name] = model_objs[name].open_derived_dataset(
                'fluxes_za_ts_monthly', 
                persist=True,
                clobber=False,
                return_none=True,
            )

CAMSv20r1
CT2017
CT2019B
CTE2018
CTE2020
MIROC
TM5-Flux-m0f
TM5-Flux-mmf
TM5-Flux-mrf
TM5-Flux-mwf
s99oc_ADJocI40S_v2020
s99oc_SOCCOM_v2020
s99oc_v2020
CPU times: user 510 µs, sys: 1.34 ms, total: 1.85 ms
Wall time: 2.42 ms


In [10]:
for model in model_list:
    if model in ['CT2019B']:
        model_objs[model].open_derived_dataset(
            'molefractions_surface_daily',
            persist=True,
            clobber=False,            
            return_none=True,                    
        )
        model_objs[model].open_derived_dataset(
            'spo_ts_daily',            
            persist=True,
            clobber=False,           
            return_none=True,                    
        )
        model_objs[model].open_derived_dataset(
            'molefractions_theta_bins',
            kwargs_name='SO-10K-bins-300K_275K',
            lat_bounds=(-80., -45.),
            theta_bins=[(295., 305.), (270., 280.),],
            persist=True,            
            clobber=False,
            return_none=True,        
        )
        model_objs[model].open_derived_dataset(
            'molefractions_theta_bins_sectors',
            kwargs_name='SO-10K-bins-300K_275K',
            lat_bounds=(-80., -45.),
            theta_bins=[(295., 305.), (270., 280.),],
            persist=True,            
            clobber=False,
            return_none=True,  
        )
        model_objs[model].open_derived_dataset(
            'molefractions_z_za', 
            persist=True,
            clobber=False,
            return_none=True,                    
        )

In [11]:
cluster.close()
client.close()